# Imports

In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [2]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path

from scipy.signal import savgol_filter

In [69]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/1.2/data_fct_Adrien_point_by_point

matplotlib.rcParams.update({'font.size': 21})

In [4]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Choose and load data

In [32]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=0)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

address_runs = [x for x in all_subdir if 'DC01_RF08' in x]
print(address_runs)

> Répertoire : /home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try00
> Filename : Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [0, 5, 12, 21, 30, 39, 49]
> slashcond | 5
No points deleted because they were not complete
Total number of data directories 8
[]


In [33]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC07_RF09', 'DC18_RF10']
000 > DC07_RF09
001 > DC18_RF10
> condition names ['DC', 'RF']
> number of points 2
> N_ions = 1024
> e_GMol = 50


In [34]:
## Data loading (1D arrays)
# select data with init to reuse
forcetryinit = ['Try00']

data_name, num_runs \
= data_retrieve_RFRelax(all_subdir,points_and_coord,condition_parameters,slash_cfg,
                        mode='RFrelax') # forcecondnum = 0 , forcetryinit = forcetryinit

data_address = data_name[0]
data0 = data_name[1]
data6 = data_name[2]

> Points | 2
> Simulations pour chaque point | ['Try00', 'Try01', 'Try02', 'Try03']
['/home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try00', '/home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try01', '/home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try02', '/home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try03', '/home/adrian/RemoteFS/Rivendel/20201202/DC18_RF10/Try00', '/home/adrian/RemoteFS/Rivendel/20201202/DC18_RF10/Try01', '/home/adrian/RemoteFS/Rivendel/20201202/DC18_RF10/Try02', '/home/adrian/RemoteFS/Rivendel/20201202/DC18_RF10/Try03']
Hello
Point n° 0
00 - 00  >  /home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try00\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG
00 - 01  >  /home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try01\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG
00 - 02  >  /home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try02\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG
00 - 03  >  /home/adrian/RemoteFS/Rivendel/20201

# load context variables

In [36]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

1.0


In [37]:
## Data loading with hardcode names
eta = 1.5e-20

plot_point = 0
plot_try = 0
# str_eta_chiffre    = float(f'{eta[plot_point]:1.2e}'[:4])/10
# str_eta_puissance  = int(re.sub('e','D',f'{eta[plot_point]:1.2e}'[6:]))-1
# str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
str_eta = '0.15D-19'
dirname_hc = str(data0[plot_point])+'/'
# Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
data0_hc    = '/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data2_hc    = '/Temp_SimuType2_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'

print(dirname_hc)
print(datainit_hc)

['/home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try00\\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG', '/home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try01\\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG', '/home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try02\\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG', '/home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try03\\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG']/
Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5


In [38]:
# recover duration for each step of simu (i_free__fly, i_laser_fly ...)
plot_try = 1
address = all_subdir[plot_point]
print(address)
info_files = sort( [f for f in listdir(address) if isfile(join(address, f)) and ".info" in f] )
print(sort(info_files))


tmp_info = loadtxt(all_subdir[plot_point]+'/'+info_files[0],encoding ='utf-8',
            skiprows=1, usecols=(0),
            unpack=True)
eta = tmp_info[-2]
i_free__fly_Lan = tmp_info[-1]

tmp_info = loadtxt(all_subdir[0]+'/'+'xva_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.info',encoding ='utf-8',
            skiprows=0, usecols=(0),
            unpack=True)
i_laser_fly_init = tmp_info[-3]
i_laser_fly_quench = tmp_info[-2]
i_relax_fly = tmp_info[-7]

n_dt = 100
dt = 1/2e6

print('eta',eta)
print('i_free__fly_Lan',i_free__fly_Lan)
print('i_laser_fly_quench',i_laser_fly_init)
print('i_laser_fly_quench',i_laser_fly_quench)
print('i_relax_fly',i_relax_fly)

/home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try00
['Langevin_cooling.info'
 'xva_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.info'
 'xva_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.info']
eta 1.5e-20
i_free__fly_Lan 1000.0
i_laser_fly_quench 6000.0
i_laser_fly_quench 10000.0
i_relax_fly 75000.0


In [60]:
plot_point = 0
plot_try = 2
data_address_aux = data_address[plot_point][plot_try]
snap_files = [f for f in listdir(data_address_aux) if isfile(join(data_address_aux, f)) and "xN" in f]
str_load = data_address_aux + '/' + snap_files[0]
print(str_load)

fid = open(str_load, 'rb')

junk0 = fromfile(fid, int32, 1)  # Read record start tag
aux = fromfile(fid, int32, 1)
junk1 = fromfile(fid, int32, 1)  # Read record stop tag
jmax = aux[0]

junk2 = fromfile(fid, int32, 1)  # Read record start tag
aux = fromfile(fid, int32, 1)
junk3 = fromfile(fid, int32, 1)  # Read record stop tag
N_ions = aux[0]

junk4 = fromfile(fid, int32, 1)  # Read record start tag
r = fromfile(fid, single, int(junk4[0]/4)) # jmax*N_ions*3
junk5 = fromfile(fid, int32, 1)  # Read record stop tag

print(jmax,N_ions)
print(junk0[0],junk1[0],junk2[0],junk3[0],junk4[0],junk5[0])
print(junk4[0]/4)

/home/adrian/RemoteFS/Rivendel/20201202/DC07_RF09/Try02/xN_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.bin
74257 1024
4 4 4 4 43008000 43008000
10752000.0


In [64]:
i_snap_x = 500
di_snap_x = 10000

snap_num = int(i_relax_fly/real(i_snap_x+di_snap_x))
snap_time = []
for k in range(1,snap_num+1):
    snap_time.append(linspace(di_snap_x*k*dt,(di_snap_x*k+i_snap_x)*dt,i_snap_x)*1000)
print(shape(snap_time))

(7, 500)


In [65]:
import matplotlib.patches as patches

In [85]:
# Get T and fluo PM  All SimuType
plot_point = 1
plot_try = 3
address_reuseinit = all_subdir[plot_point]
address = data_address[plot_point][plot_try]
init_data = data0[plot_point][plot_try]
evol_data = data6[plot_point][plot_try]
print(init_data)
print(evol_data)

# onlyfiles = [f for f in listdir(evol_data) if isfile(join(evol_data, f)) and "SimuType" in f and ".dat" in f]
# print(sort(onlyfiles))
# onlyfiles_reuseinit = [f for f in listdir(init_data) if isfile(join(init_data, f)) and "SimuType" in f and ".dat" in f]
# print(sort(onlyfiles_reuseinit))

ta, te, ti = plot_T_and_PM_InitQ_Evol_AfterCool(address,
                                           flag_plot=0,fig_name='0o0o0o0',xlim1=(0,65),ylim1=(2e-4,6000),ylim2=(-50,46000))

fname = address[31:49]
plt.figure(re.sub('/','-',fname)).clf()
ax1 = subplot(111)
te_filter = []
for k in range(3):
    te_filter.append(savgol_filter(te[:,k], 25, 1))
    ax1.semilogy(ta*1e3,te_filter[k]) ; ax1.text(0,1e-4,'Filtered temperatures',fontsize = 10,ha='left')
    # ax1.semilogy(ta*1e3,te[:,k])
    ax1.grid()
    
ax1.set_xlabel('time [µs]')
ax1.set_ylabel('T [K]')
title(fname)
    
time_switch = i_free__fly_Lan *dt
ax1.vlines(time_switch*1e3,2e-4,1e4,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly_init)*dt
ax1.vlines(time_switch*1e3,2e-4,1e4,color='green')
time_switch = (i_free__fly_Lan+i_laser_fly_init+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,2e-4,1e4,color='orange')
for k in range(snap_num):
    ax1.add_patch(
     patches.Rectangle(
        (snap_time[k][0]+time_switch*1e3, 1e-4),
        snap_time[k][-1]-snap_time[k][0],
        0.01,
        edgecolor = 'blue',
        facecolor = 'red',
        fill=True
     ) )
time_switch = (i_free__fly_Lan+i_laser_fly_init+i_laser_fly_quench+i_relax_fly)*dt
ax1.vlines(time_switch*1e3,2e-4,1e4,color='purple')

tight_layout()

# tt2, T_CM2, T_aux2, PM2
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

/home/adrian/RemoteFS/Rivendel/20201202/DC18_RF10/Try03\Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20201202/DC18_RF10/Try03\Temp_SimuType6_01_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
len save_T [55440]
[5.05000000e-07 2.93594610e-06 7.61476188e-04 4.42512894e-05
 4.83924365e+01 1.50717352e+00 6.05032267e+01]


In [86]:
r_reshape = reshape(r,(N_ions,3,int(junk4[0]/4/3/N_ions)),order='F')
shape(r_reshape)

(1024, 3, 3500)

In [103]:
r_reshape[:,0,500]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [93]:
figure('my_little_cloud').clf()
time = 1000
for k in range(1024):
    scatter(r_reshape[k,0,time]*1000,r_reshape[k,1,time]*1000)

In [225]:
figure('my_little_cloud').clf()
for time in range(0,2000,200):
    for k in range(1024):
        scatter(r_reshape[k,0,time]*1000,r_reshape[k,1,time]*1000)
    plt.pause(1)
    plt.clf()

3.4982388e-06